## Employee_Number
- 1080 valores únicos (incluyendo el 0)
- 534 duplicados
- (1080 + 534 = 1614 totales)
- ...................
- dtype  -> object

## Duplicados

- 431 son 'Employee_Number'= 0
- 104 en las siguientes situaciones

- df_duplicados = 639 rows -> 431 'Employee_Number'=0 + (104 x 2)


- Situación 1: 51 employees
    - Todas las columnas son iguales
   
    ➡️ Solución: eliminar el primer registro   

- Situación 2: 16 employees
    - Distance_From_Home diferentes
   
    ➡️ Solución: eliminar el primer registro   

- Situación 3: 48 employees
    - Remote_Work diferentes
   
    ➡️ Solución: eliminar el primer registro  

- Situación 4: 6 employees
    - Distance_From_Home diferentes
    - Remote_Work diferentes

    ➡️ Solución: eliminar el primer registro

- Situación 5: 8 employees
    - Hourly_Rate -> Not Available
   
    ➡️ Solución: eliminar Hourly_Rate == Not Available     

- Situación 6: 0 employees
    - Distance_From_Home diferentes
    - Distance_From_Home = Environment_Satisfaction
   
    ➡️ Solución: eliminar Distance_From_Home = Environment_Satisfaction

- Situación 7: 7 employees
    - Distance_From_Home diferentes
    - Environemnt_Satisfaction diferentes

      ➡️ Solución: eliminar el primer registro


- Situación 8: 6 employees
    - Distance_From_Home diferentes
    - Environemnt_Satisfaction diferentes
    - Remote_Work

      ➡️ Solución: eliminar el primer registro


In [98]:
# Importamos librerias
import pandas as pd
import numpy as np

import re

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Ajustes de visualización
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

In [99]:
# Leemos el .csv y lo guardamos en una variable df
def leer_fichero(url):

    """
    Esta función abre un fichero .csv a partir de la dirección (url) especificad y genera un DataFrame.

    Parameters:
    -----------------
    url (str): String con la dirección del fichero .csv que queremos abrir.

    Returns:
    ------------
    df_original (DataFrame): El fichero en formato DataFrame
    """
    
    df_original = pd.read_csv(url, index_col=0)

    return df_original

## Paso 1: Limpieza de datos

### 1.1. Modificar nombre columnas


In [113]:
def cambiar_nombre_col(data):

    """
    Esta función modifica los nombres de todas las columnas para que queden en formato Xxxx_Xxxx.

    Parameters:
    -----------------
    data (DataFrame): El DataFrame al que le queremos modificar los nombres de las columnas.

    Returns:
    ------------
    returns: data (DataFrame).
    """
     
    def normalizar_encabezados(encabezado):

        """
        Esta función modifica el formato del texto para que quede en formato Xxxx_Xxxx.

        Parameters:
        -----------------
        encabezado (str): El texto al que le queremos cambiar el formato.

        Returns:
        ------------
        encabezado: El texto con el formato cambiado.
        """

        # Mapeo para palabras completamente en mayúsculas pq no encuentro patrón para dividir.
        casos_especiales = {
            'NUMCOMPANIESWORKED': 'Num_Companies_Worked',
            'TOTALWORKINGYEARS': 'Total_Working_Years',
            'YEARSWITHCURRMANAGER': 'Years_With_Curr_Manager',
            'NUMBERCHILDREN': 'Number_Children', # Añado otros casos especiales, Los pongo en mayúsculas para que los trate trate igual a los anteriores.
            'EMPLOYEECOUNT': 'Employee_Count',
            'EMPLOYEENUMBER': 'Employee_Number',
            'WORKLIFEBALANCE': 'Work_Life_Balance'
        }

        # Si el encabezado está en casos especiales devolver el formato dado.
        if encabezado.upper() in casos_especiales:
            return casos_especiales[encabezado.upper()]

        # Añadir guion bajo entre letras y números
        encabezado = re.sub(r'([a-zA-Z])(\d)', r'\1_\2', encabezado)

        # Para formatos que incluyen transiciones de minúscula a mayúscula
        if re.match(r'^[A-Za-z0-9]+$', encabezado):  # Si son letras seguidas de números sin guiones
            encabezado = re.sub(r'([a-z])([A-Z])', r'\1_\2', encabezado)
            encabezado = re.sub(r'([A-Z]+)([A-Z][a-z]+)', r'\1_\2', encabezado)

    # Capitalizar cada string separado por guiones
        palabras = encabezado.split('_')
        return '_'.join(word.capitalize() for word in palabras)

    data.columns = [normalizar_encabezados(columna) for columna in data.columns]
    return data


### 1.2. Eliminar columnas no necesarias

In [4]:

def eliminar_columnas(data, lista_col):

    """
    Esta función elimina columnas (lista_col) de un DataFrame (data).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame al que le queremos eliminar columnas.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos eliminar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """
            
    for col in lista_col:
        data.drop(col, axis=1, inplace=True)

    return data

### 1.3. Crear columnas nuevas

In [5]:
def crear_columnas(data, *lista_col):

    """
    Esta función crea/añade columnas (lista_col) en un DataFrame (data).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame al que le queremos añadir columnas.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos añadir.

    Returns:
    ------------
    data: El DataFrame modificado.
    """
        
    for col in lista_col:
        data.insert(1,'Age_Group', ' ')

    return data

### 1.4. Modificar valores en columnas

1.4.1 Convertir a str

In [6]:
def to_string(data, lista_col):

    """
    Esta función cambia el tipo de datos de una columna a string (str).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """
    
    for col in lista_col:
        data[col] = data[col].astype(str)

    return data

   

1.4.2 Cambio de ',' por '.'

In [7]:
# lista_col_to_punto=['Daily_Rate']

# for col in lista_col_to_punto:
#     df[col] = df[col].str.replace(',','.')

1.4.3 Cambio de ',0' por ''

In [8]:
def to_no_coma(data, lista_col):

    """
    Esta función elimina los ',0' de los valores de la columa especificada en un DataFrame (data). 
    El formato de los valores en la columna a modificar tienen que ser string (str).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for col in lista_col:
        data[col] = data[col].str.replace(',0','')
    return data

1.4.4 Booleanos

In [9]:
def to_bool(data,lista_col):

    """
    Esta función modifica los datos de una columna de tipo booleano (bool) de un DataFrame (data).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """
    
    dict_booleanos = {'1':True, 
                    'Yes':True, 
                    'True':True, 
                    '0':False, 
                    'No':False,
                    'False':False}


    for col in lista_col:
        data[col] = data[col].replace(dict_booleanos)

    return data

1.4.5 Otros cambios

In [10]:
def limp_otros(data,lista_col):

    """
    Esta función realiza cambios varios a las columnas especificadas en un DataFrame (data).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for col in lista_col:

        if col == 'Age':
            reemplazos = {
                'forty-seven': 47,
                'fifty-eight': 58,
                'thirty-six': 36,
                'fifty-five': 55,
                'fifty-two': 52,
                'thirty-one': 31,
                'thirty': 30,
                'twenty-six': 26,
                'thirty-seven': 37,
                'thirty-two': 32,
                'twenty-four': 24
            }
            data[col] = data[col].replace(reemplazos)

        elif col == 'Business_Travel':
            data[col] = data[col].str.replace('-',' ').str.replace('_',' ')

        elif col == 'Daily_Rate':
            data[col] = data[col].str.replace('$', '')

        elif col == 'Department':
            data[col]=data[col].str.strip()
            
            for index, row in data.iterrows():
                job_role = row['Job_Role'].strip().lower()
                if job_role == 'sales executive':
                    data.loc[index, col] = 'Sales'
                if job_role == 'human resourses':
                    data.loc[index, col] = 'Human Resources'
                if job_role == 'research director'  or job_role == 'manufacturing director' or job_role == 'laboratory technician'or job_role == 'research scientist'or job_role == 'healthcare representative':
                    data.loc[index, col] = 'Research & Development'

        elif col == 'Distance_From_Home':
            data[col] = data[col].str.replace('-', '')

        elif col == 'Environment_Satisfaction':
            for index, row in data.iterrows():
                numero = int(row[col])
                if numero < 10 and numero not in [2, 3, 4]:
                    data.loc[index, col] = 1
                elif 10 <= numero <= 20:
                    data.loc[index, col] = 2
                elif 20 < numero <= 30:
                    data.loc[index, col] = 3
                elif 30 < numero <= 49:
                    data.loc[index, col] = 4

        elif col == 'Gender':
            data[col] = data[col].replace({'0': 'M', '1': 'F'})

        elif col == 'Marital_Status':
            data[col] = data[col].replace({'Marreid':'Married'})

        else:
            data[col] = data[col] / 100

    return data

1.4.6 Str a .title()

In [11]:
def to_title(data,lista_col):

    """
    Esta función modifica los valores de una columna que tenga formato string (str) y los modifica para que tengan las primeras letras en mayúscula.

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """
    
    for col in lista_col:
        data[col] = data[col].str.title()

    return data

1.4.7 Cambio de nulos str a 99999


In [12]:
def to_int_nulos(data,lista_col):

    """
    Esta función cambia los valores nulos de la columa especificada que este en formato string (str) en un DataFrame (data) y los reemplaza por '99999'.

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for col in lista_col:
        data[col]=data[col].replace({'nan':99999,'NaN':99999, 'Not Available':99999})

    return data


## 1.5 Cambio a formato deseado

1.5.1 Cambio a int

In [13]:
def to_int(data,lista_col):

    """
    Esta función cambia el tipo de datos de una columna a integer (int).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for col in lista_col:
        data[col] = data[col].astype(int)
    return data

1.5.2 Cambio a float

In [14]:
# lista_col_to_float=['Daily_Rate']

# for col in lista_col_to_float:
#     df[col] = df[col].astype(float)

1.5.3 Cambio a Bool

In [15]:
def to_bool(data,lista_col):

    """
    Esta función cambia el tipo de datos de una columna a booleano (bool).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """
        
    for col in lista_col:
        data[col] = data[col].astype(bool)
    return data

## 1.6 Reemplazando nulos

1.6.1 Int con .median()

In [16]:
def to_median(data,lista_col):

    """
    Esta función reemplaza los nulos de una columna que esta en formnato integer (int) por el valor de la mediana (median).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for col in lista_col:
        data[col]=data[col].replace({99999:np.nan})
        data[col] = data[col].fillna(data[col].median()).astype(int)

    return data

1.6.2 int con .mean()

In [17]:
def to_mean(data,lista_col):

    """
    Esta función reemplaza los nulos de una columna que esta en formnato integer (int) por el valor de la media (mean).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for col in lista_col:
        data[col]=data[col].replace({99999:np.nan})
        data[col] = data[col].fillna(data[col].mean()).astype(int)
    return data

1.6.4 Categóricos a 'Unknown'

In [18]:
# creo la funcion
def reemplazar_nulos_por_unknown(data, lista_col):

    """
    Esta función reemplaza los nulos de una columna que esta en formnato object por el valor 'Unknown''.

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_col (lista): Lista de strings con el nombre de las columnas que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for columna in lista_col:
        data[columna] = data[columna].replace({'nan':None,'Nan':None})
        data[columna].fillna('Unknown', inplace=True)
    return data

### 1.7 Rellenar columnas nuevas

In [19]:
#Columna 'Age_Group'
def age_group(data, col):

    """
    Esta función introduce datos en la columa 'Age_Group'

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    col (str): String con el nombre de la columna que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    rangos_edad = ['18-25', '26-35', '36-45', '46-55', '56-65']
    limites = [18, 26, 36, 46, 56, 66] # define los límites de cada grupo de edad, donde cada límite es exclusivo del grupo siguiente (excepto el último).


    data[col] = pd.cut(data['Age'], bins=limites, labels=rangos_edad, right=False)
    return data

In [42]:
# Columna Satisfaction_Group
def asignar_grupo(data, job_satisfaction):

    """
    Esta función introduce datos en la columa 'Job_Satisfaction'

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    job_satisfaction (str): String con el nombre de la columna que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    for index, row in data.iterrows():
        if row[job_satisfaction] >= 3:
            data.loc[index,'Satisfaction_Group']= 'A'
        else:
            data.loc[index,'Satisfaction_Group']= 'B'
    return data

## Paso 2:Eliminamos duplicados

In [45]:
#Obtenemos el listado del nombre de las columnas del DF
lista_col_names = set(df.columns.tolist())
len(lista_col_names)

34

In [46]:
#Obtenemos un df de los duplicados
df_duplicados = df[df['Employee_Number'].isin(df['Employee_Number'][df['Employee_Number'].duplicated()])].sort_values('Employee_Number')
df_duplicados

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Department,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Rate,Num_Companies_Worked,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work,Satisfaction_Group
82,38,36-45,True,Unknown,119,Sales,3,3,Life Sciences,307,1,M,76,3,3,Sales Executive,3,Divorced,9647,0,True,0.12,3,3,2,11,6,2,16,5,13,1985,True,A
1474,38,36-45,True,Unknown,119,Sales,3,3,Life Sciences,307,1,M,76,3,3,Sales Executive,3,Divorced,9647,0,True,0.12,3,3,2,11,6,2,16,5,13,1985,True,A
1476,55,46-55,True,Travel Rarely,452,Unknown,1,3,Medical,374,4,M,81,3,5,Manager,1,Single,18938,0,True,0.14,3,3,0,37,2,3,36,4,13,1968,True,B
84,55,46-55,True,Travel Rarely,452,Unknown,1,3,Medical,374,4,M,81,3,5,Manager,1,Single,18938,0,True,0.14,3,3,0,37,2,3,36,4,13,1968,True,B
1542,47,46-55,True,Non Travel,666,Unknown,29,4,Life Sciences,376,1,M,88,3,3,Manager,2,Married,10268,1,True,0.12,3,4,1,11,2,2,10,9,9,1976,True,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,35,26-35,True,Travel Rarely,528,Unknown,8,4,Technical Degree,99999,3,M,100,3,1,Human Resources,3,Unknown,7108,1,True,0.17,3,2,0,6,2,1,5,1,4,1988,True,A
610,54,46-55,True,Unknown,1082,Sales,2,4,Life Sciences,99999,3,F,41,2,3,Sales Executive,3,Married,8392,6,True,0.11,3,2,1,13,4,3,9,7,0,1969,True,A
607,30,26-35,True,Travel Rarely,231,Sales,8,2,Unknown,99999,3,M,62,3,3,Sales Executive,3,Unknown,9977,5,True,0.11,3,1,1,11,2,4,8,7,7,1993,True,A
661,31,26-35,True,Non Travel,99999,Research & Development,5,3,Medical,99999,2,M,74,3,2,Manufacturing Director,1,Unknown,3787,0,True,0.19,3,2,0,10,2,3,9,1,7,1992,True,B


In [47]:
#Comprobamos cuantos duplicados tenemos en 'Employee_Number' 
df['Employee_Number'].duplicated().sum()

534

In [48]:
#Comprobamos la cantidad que tenemos de 'Employee_Number'==99999
df[df['Employee_Number']==99999].shape[0]

431

### 2.1 Situación 1: Todas las columnas son iguales

In [49]:
#Eliminamos los duplicados 
def eliminar_duplicados (data,subset):

    """
    Esta función elimina duplicados de un DataFrame (data) utilizando un subset formado por nombres de las columnas que queremos comparar.

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    subset (lista): Lista de strings con los nombres de la columnas que queremos comparar para ver si hay duplicados.

    Returns:
    ------------
    data: El DataFrame modificado.
    """


    data.drop_duplicates(subset=subset,keep='last',inplace=True)

    return data

In [50]:
#Comprobamos el número de duplicados en el DF
df.duplicated().sum()

99

In [51]:
df = eliminar_duplicados(df,lista_col_names)

In [52]:
#Comprobamos el número de duplicados en el DF
df.duplicated().sum()

0

### 2.2 Situación 2:Columna 'Distance_From_Home' diferente

In [54]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_2 = set(lista_col_names)
lista_columnas_2.remove('Distance_From_Home')
len(lista_columnas_2)

33

In [55]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_2].duplicated().sum()

22

In [56]:
#Eliminamos los duplicados del subset
# df.drop_duplicates(subset=lista_columnas_2, keep='last',inplace=True)
df = eliminar_duplicados(df,lista_columnas_2)

In [57]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_2].duplicated().sum()

0

### 2.3 Situación 3: Columnas 'Remote_Work' diferente

In [58]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_3 = set(lista_col_names)
lista_columnas_3.remove('Remote_Work')
len(lista_columnas_3)

33

In [59]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_3].duplicated().sum()

0

In [60]:
#Eliminamos los duplicados del subset
#df.drop_duplicates(subset=lista_columnas_3,keep='last',inplace=True)
df = eliminar_duplicados(df,lista_columnas_3)

In [61]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_3].duplicated().sum()

0

### 2.4 Situación 4: Columna 'Distance_From_Home' & 'Remote_Work' diferentes

In [62]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_4 = set(lista_col_names)
lista_columnas_4.remove('Distance_From_Home')
lista_columnas_4.remove('Remote_Work')
len(lista_columnas_4)

32

In [63]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_4].duplicated().sum()

0

In [64]:
#Eliminamos los duplicados del subset
#df.drop_duplicates(subset=lista_columnas_4,keep='last',inplace=True)
df = eliminar_duplicados(df,lista_columnas_4)

In [65]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_4].duplicated().sum()

0

### 2.5 Situación 5: Columna 'Hourly_Rate' == 'Not_Available'

In [66]:
# Creamos un DF de duplicados nuevo
df_duplicados_s5 = df[df['Employee_Number'].isin(df['Employee_Number'][df['Employee_Number'].duplicated() ])].sort_values('Employee_Number')
df_duplicados_s5

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Department,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Rate,Num_Companies_Worked,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work,Satisfaction_Group
871,31,26-35,True,Travel Rarely,691,Research & Development,11,4,Unknown,423,2,M,86,3,1,Research Scientist,4,Married,10077,0,True,0.12,3,3,1,11,4,3,5,0,3,1992,True,A
1512,31,26-35,True,Travel Rarely,691,Research & Development,5,4,Unknown,423,3,M,86,3,1,Research Scientist,4,Married,10077,0,True,0.12,3,3,1,11,4,3,5,0,3,1992,True,A
1515,52,46-55,True,Unknown,723,Research & Development,8,4,Medical,433,3,M,85,2,2,Research Scientist,2,Unknown,17747,2,True,0.15,3,1,0,11,3,2,8,7,7,1971,True,B
874,52,46-55,True,Unknown,723,Research & Development,17,4,Medical,433,2,M,85,2,2,Research Scientist,2,Unknown,17747,2,True,0.15,3,1,0,11,3,2,8,7,7,1971,True,B
1527,29,26-35,True,Unknown,1283,Research & Development,23,3,Unknown,495,4,M,54,3,1,Research Scientist,4,Unknown,18168,9,True,0.16,3,4,0,6,4,3,3,1,2,1994,True,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,35,26-35,True,Travel Rarely,1214,Research & Development,1,3,Medical,99999,2,M,30,2,1,Research Scientist,3,Single,26278,1,True,0.18,3,1,0,6,3,3,6,0,4,1988,True,A
561,32,26-35,True,Travel Rarely,548,Research & Development,1,3,Life Sciences,99999,2,M,66,3,2,Research Scientist,2,Unknown,7346,1,True,0.17,3,2,2,10,3,3,10,0,9,1991,True,B
559,35,26-35,True,Travel Rarely,809,Research & Development,16,3,Medical,99999,1,M,84,4,1,Laboratory Technician,2,Married,16479,0,True,0.13,3,3,1,6,5,3,5,0,3,1988,True,B
904,29,26-35,True,Unknown,408,Research & Development,25,5,Technical Degree,99999,3,F,71,2,1,Research Scientist,2,Married,18300,5,True,0.16,3,2,0,6,2,4,2,1,1,1994,True,B


In [67]:
# Filrtamos el DF para obtener el DF dónde el 'Hourly_Rate'==99999 y el 'Employee_Number' != 999999
df_duplicados_s5_filtrado = df_duplicados_s5[df_duplicados_s5['Hourly_Rate']==99999]
df_duplicados_s5_filtrado = df_duplicados_s5_filtrado.loc[df_duplicados_s5_filtrado['Employee_Number']!=99999,:]
df_duplicados_s5_filtrado


,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Department,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Rate,Num_Companies_Worked,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work,Satisfaction_Group
892,42,36-45,True,Unknown,201,Research & Development,1,4,Unknown,517,2,F,99999,3,1,Laboratory Technician,1,Unknown,20490,3,True,0.16,3,2,1,8,5,3,5,1,2,1981,True,B
1566,35,26-35,True,Travel Rarely,1276,Research & Development,16,3,Life Sciences,586,4,M,99999,3,3,Healthcare Representative,3,Unknown,14295,4,True,0.12,3,3,0,10,2,3,8,0,0,1988,True,A
1578,29,26-35,True,Unknown,806,Research & Development,1,4,Life Sciences,710,2,M,99999,1,1,Research Scientist,4,Divorced,18959,1,True,0.18,3,4,1,10,5,3,10,2,8,1994,True,A
340,27,26-35,True,Travel Rarely,99999,Research & Development,1,2,Life Sciences,725,3,F,99999,3,3,Manufacturing Director,1,Married,6009,1,True,0.11,3,4,0,9,3,3,9,0,7,1996,True,B
1604,41,36-45,True,Unknown,1085,Research & Development,2,4,Life Sciences,927,2,F,99999,1,1,Laboratory Technician,4,Divorced,17725,4,True,0.13,3,3,1,11,1,2,7,1,0,1982,True,A
1605,31,26-35,True,Travel Rarely,154,Unknown,7,4,Unknown,941,2,M,99999,2,1,Sales Representative,3,Unknown,11737,3,True,0.15,3,2,0,11,2,4,7,5,2,1992,True,A
987,33,26-35,True,Non Travel,1038,Sales,8,1,Life Sciences,1044,2,F,99999,2,1,Sales Representative,4,Unknown,21437,0,True,0.19,3,4,0,11,2,2,2,2,2,1990,True,A
1008,35,26-35,True,Travel Frequently,99999,Research & Development,4,4,Other,1185,4,M,99999,2,1,Laboratory Technician,4,Married,26537,1,True,0.13,3,2,1,2,2,4,2,2,2,1988,True,A


In [68]:
#Guardamos en una lista los números de los índices de todas las filas del DF filtrado 'df_duplicados_s5_filtrado'
lista_index_situacion_5= df_duplicados_s5_filtrado.index.tolist()
len(lista_index_situacion_5)

8

In [1]:
def eliminar_por_index (data, lista_index):

    """
    Esta función elimina duplicados de un DataFrame (data) utilizando una lista con los indices de las filas a eliminar (lista_index).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    lista_index (lista): Lista de strings con los nombres de los índices de las filas que queremos eliminar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    
    for employee in lista_index:
        data.drop(employee,axis=0,inplace=True)

    return data

In [70]:
#Utilizamos un for loop para iterar por la lista de los números de índice y eliminamos cada uno de estos de nuestro DF original
# for employee in lista_index_situacion_5:
#     df.drop(employee,axis=0,inplace=True)

df = eliminar_por_index(df, lista_index_situacion_5)

In [71]:
df['Employee_Number'].duplicated().sum()

405

### 2.6 Situación 6: Columna 'Distance_From_Home' == 'Environment_Satisfaction'

In [72]:
# Creamos un DF de duplicados nuevo
df_duplicados_s6 = df[df['Employee_Number'].isin(df['Employee_Number'][df['Employee_Number'].duplicated() ])].sort_values('Employee_Number')
df_duplicados_s6

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Department,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Rate,Num_Companies_Worked,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work,Satisfaction_Group
871,31,26-35,True,Travel Rarely,691,Research & Development,11,4,Unknown,423,2,M,86,3,1,Research Scientist,4,Married,10077,0,True,0.12,3,3,1,11,4,3,5,0,3,1992,True,A
1512,31,26-35,True,Travel Rarely,691,Research & Development,5,4,Unknown,423,3,M,86,3,1,Research Scientist,4,Married,10077,0,True,0.12,3,3,1,11,4,3,5,0,3,1992,True,A
1515,52,46-55,True,Unknown,723,Research & Development,8,4,Medical,433,3,M,85,2,2,Research Scientist,2,Unknown,17747,2,True,0.15,3,1,0,11,3,2,8,7,7,1971,True,B
874,52,46-55,True,Unknown,723,Research & Development,17,4,Medical,433,2,M,85,2,2,Research Scientist,2,Unknown,17747,2,True,0.15,3,1,0,11,3,2,8,7,7,1971,True,B
886,29,26-35,True,Unknown,1283,Research & Development,17,3,Unknown,495,2,M,54,3,1,Research Scientist,4,Unknown,18168,9,True,0.16,3,4,0,6,4,3,3,1,2,1994,True,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,35,26-35,True,Travel Rarely,809,Research & Development,16,3,Medical,99999,1,M,84,4,1,Laboratory Technician,2,Married,16479,0,True,0.13,3,3,1,6,5,3,5,0,3,1988,True,B
558,41,36-45,True,Unknown,1102,Sales,1,2,Unknown,99999,2,F,94,3,2,Sales Executive,4,Unknown,19479,8,True,0.11,3,1,0,8,0,1,6,0,5,1982,True,A
557,47,46-55,True,Unknown,1093,Sales,9,3,Unknown,99999,3,M,82,1,4,Sales Executive,3,Married,24164,7,True,0.11,3,3,0,25,3,1,23,14,10,1976,True,A
591,52,46-55,True,Travel Rarely,319,Research & Development,32,3,Unknown,99999,4,M,99999,2,3,Manufacturing Director,3,Married,19609,2,True,0.14,3,3,0,11,4,3,5,0,4,1971,True,A


In [73]:
# Filtramos el DF para obtener el DF dónde el 'Distance_From_Home' == 'Environment_Satisfaction'
# y el 'Employee_Number' != 999999
df_duplicados_s6_filtrado = df_duplicados_s6[(df_duplicados_s6['Distance_From_Home']==df_duplicados_s6['Environment_Satisfaction'])]
df_duplicados_s6_filtrado = df_duplicados_s6_filtrado.loc[df_duplicados_s6_filtrado['Employee_Number']!=99999,:]
df_duplicados_s6_filtrado

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Department,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Rate,Num_Companies_Worked,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work,Satisfaction_Group


In [74]:
# Guardamos en una lista los números de los índices de todas las filas del DF filtrado 'df_duplicados_s6_filtrado'
lista_index_situacion_6=df_duplicados_s6_filtrado.index.tolist()
len(lista_index_situacion_6)

0

In [75]:
#Utilizamos un for loop para iterar por la lista de los números de índice y eliminamos cada uno de estos de nuestro DF original
# for employee in lista_index_situacion_6:
#     df.drop(employee,axis=0,inplace=True)

df = eliminar_por_index(df, lista_index_situacion_6)

### 2.7 Situación 7: Columna 'Distance_From_Home' y 'Environment_Satisfaction' distintas

In [76]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_7 = set(lista_col_names)
lista_columnas_7.remove('Distance_From_Home')
lista_columnas_7.remove('Environment_Satisfaction')
len(lista_columnas_7)

32

In [77]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_7].duplicated().sum()

13

In [78]:
#Eliminamos los duplicados del subset
#df.drop_duplicates(subset=lista_columnas_7,keep='last',inplace=True)
df=eliminar_duplicados(df, lista_columnas_7)

In [79]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_7].duplicated().sum()

0

### 2.8 Situación 8: Columna 'Distance_From_Home' y 'Environment_Satisfaction' distintas y 'Remote_Work'

In [80]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_8 = set(lista_col_names)
lista_columnas_8.remove('Distance_From_Home')
lista_columnas_8.remove('Environment_Satisfaction')
lista_columnas_8.remove('Remote_Work')
len(lista_columnas_8)

31

In [81]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_8].duplicated().sum()

0

In [82]:
#Eliminamos los duplicados del subset
#df.drop_duplicates(subset=lista_columnas_8,keep='last',inplace=True)
df = eliminar_duplicados(df, lista_columnas_8)

In [83]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_8].duplicated().sum()

0

## Paso 3: Cambio 'Employee_Number' Nulos

In [84]:
# Mostramos del DF original los 'Employee_number' == 99999
df[df['Employee_Number']==99999]

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Department,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Rate,Num_Companies_Worked,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work,Satisfaction_Group
3,47,46-55,True,Travel Rarely,359,Research & Development,2,4,Medical,99999,1,F,82,3,4,Research Director,3,Married,26703,3,True,0.19,3,2,2,11,2,3,20,5,6,1976,True,A
4,46,46-55,True,Unknown,1319,Sales,3,3,Technical Degree,99999,1,F,45,4,4,Sales Executive,1,Divorced,7739,2,True,0.12,3,4,1,11,5,3,19,2,8,1977,True,B
8,41,36-45,True,Unknown,1276,Unknown,2,5,Unknown,99999,2,F,91,3,4,Manager,1,Married,5626,7,True,0.16,3,2,1,22,2,3,18,11,8,1982,True,B
11,56,56-65,True,Travel Rarely,1369,Research & Development,23,3,Life Sciences,99999,4,M,68,3,4,Manufacturing Director,2,Married,18235,4,True,0.12,3,1,1,33,0,3,19,15,9,1967,True,B
12,38,36-45,True,Unknown,201,Research & Development,10,3,Medical,99999,2,F,99,1,3,Research Director,3,Married,3376,3,True,0.12,3,1,1,11,3,3,18,1,11,1985,True,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601,28,26-35,True,Unknown,760,Sales,43,4,Unknown,99999,4,F,81,3,2,Sales Executive,2,Unknown,3698,1,True,0.20,4,1,0,8,2,3,8,7,5,1995,True,B
1602,26,26-35,True,Unknown,99999,Research & Development,5,2,Medical,99999,3,F,88,2,1,Research Scientist,3,Married,20898,1,True,0.14,3,1,1,8,2,3,8,1,7,1997,True,A
1607,31,26-35,True,Unknown,616,Research & Development,12,3,Medical,99999,4,F,41,3,2,Healthcare Representative,4,Unknown,17369,0,True,0.11,3,3,2,10,2,1,9,8,5,1992,True,A
1611,39,36-45,True,Travel Rarely,903,Sales,13,5,Unknown,99999,2,M,41,4,3,Sales Executive,3,Single,2560,0,True,0.18,3,4,0,9,3,3,8,0,7,1984,True,A


In [88]:
def cambio_employee_num (data):


    """
    Esta función reemplaza los valores '99999' en la columna 'Employee_Number' del DataFrame (data) por valores nuevos que se generan 
    a partir del valor más elevado de la columna 'Employee_Number'.

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    lista_employee_number = data['Employee_Number'].unique().tolist()
    lista_employee_number.sort()
    lista_employee_number.remove(99999)
    
    new_employee_number = (lista_employee_number[-1]) + 1

    indices = data.index[data['Employee_Number'] == 99999]

    #Iteramos con un for loop por la lista de índices y cambiamos el valor del 'Employee_Number' por el nuevo valor
    for i in indices:
        data.loc[i, 'Employee_Number'] = new_employee_number
        new_employee_number += 1

    return data

In [89]:
df = cambio_employee_num(df)

In [90]:
#Comprobamos el tamaño final del DF
df.shape

(1472, 34)

In [91]:
#Comprobamos si quedan duplicados en la columna de 'Employee_Number'
df['Employee_Number'].duplicated().sum()

0

In [92]:
def set_col_index (data, col):

    """
    Esta función asigna una columna (col) como columna índice de un DataFrame (data).

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos modificar.
    subset (lista): Lista de strings con los nombres de la columnas que queremos comparar para ver si hay duplicados.

    Returns:
    ------------
    data: El DataFrame modificado.
    """

    data.set_index(col,drop=True,inplace=True)

    return data

In [93]:
#Adjudicamos la columna 'Employee_Number' cómo la columna de ínidce y eliminamos el actual índice
# df.set_index('Employee_Number',drop=True,inplace=True)

df = set_col_index(df, 'Employee_Number')

In [94]:
#Visualizamos el DF resultante
df.head(5)

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Department,Distance_From_Home,Education,Education_Field,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Rate,Num_Companies_Worked,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work,Satisfaction_Group
Employee_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
162,51,46-55,True,Unknown,684,Research & Development,6,3,Unknown,1,M,51,3,5,Research Director,3,Unknown,6462,7,True,0.13,3,3,0,11,5,3,20,15,15,1972,True,A
259,52,46-55,True,Unknown,699,Unknown,1,4,Life Sciences,3,M,65,2,5,Manager,3,Unknown,5678,0,True,0.14,3,1,1,34,5,3,33,11,9,1971,True,A
319,42,36-45,True,Travel Rarely,532,Research & Development,4,2,Technical Degree,3,M,58,3,5,Manager,4,Married,4933,1,True,0.11,3,4,0,22,3,3,22,11,15,1981,True,A
2069,47,46-55,True,Travel Rarely,359,Research & Development,2,4,Medical,1,F,82,3,4,Research Director,3,Married,26703,3,True,0.19,3,2,2,11,2,3,20,5,6,1976,True,A
2070,46,46-55,True,Unknown,1319,Sales,3,3,Technical Degree,1,F,45,4,4,Sales Executive,1,Divorced,7739,2,True,0.12,3,4,1,11,5,3,19,2,8,1977,True,B


In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 162 to 2461
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Age                         1472 non-null   int64   
 1   Age_Group                   1472 non-null   category
 2   Attrition                   1472 non-null   bool    
 3   Business_Travel             1472 non-null   object  
 4   Daily_Rate                  1472 non-null   int64   
 5   Department                  1472 non-null   object  
 6   Distance_From_Home          1472 non-null   int64   
 7   Education                   1472 non-null   int64   
 8   Education_Field             1472 non-null   object  
 9   Environment_Satisfaction    1472 non-null   int64   
 10  Gender                      1472 non-null   object  
 11  Hourly_Rate                 1472 non-null   int64   
 12  Job_Involvement             1472 non-null   int64   
 13  Job_Level       

In [224]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,1472.0,36.930707,9.134501,18.00,30.00,36.00,43.00,60.00
Daily_Rate,1472.0,8353.369565,26311.506384,103.00,503.25,858.00,1239.25,99999.00
Distance_From_Home,1472.0,11.247283,10.487894,1.00,2.00,8.00,17.00,49.00
Education,1472.0,2.912364,1.023748,1.00,2.00,3.00,4.00,5.00
Environment_Satisfaction,1472.0,2.743207,1.090386,1.00,2.00,3.00,4.00,4.00
Hourly_Rate,1472.0,5225.632473,22120.663011,30.00,49.75,68.00,86.00,99999.00
Job_Involvement,1472.0,2.729620,0.711367,1.00,2.00,3.00,3.00,4.00
Job_Level,1472.0,2.065897,1.107606,1.00,1.00,2.00,3.00,5.00
Job_Satisfaction,1472.0,2.728261,1.102283,1.00,2.00,3.00,4.00,4.00
Monthly_Rate,1472.0,14313.053668,7122.366418,2094.00,8043.75,14235.50,20463.25,26999.00


In [225]:
df.describe(include=['object','category','bool']).T

,count,unique,top,freq
Age_Group,1472,5,26-35,607
Attrition,1472,2,False,1235
Business_Travel,1472,4,Unknown,706
Department,1472,4,Research & Development,921
Education_Field,1472,7,Unknown,684
Gender,1472,2,M,884
Job_Role,1472,9,Sales Executive,326
Marital_Status,1472,4,Unknown,590
Over_Time,1472,3,False,623
Satisfaction_Group,1472,2,A,902


In [95]:
#Guardamos el DF en formato .csv

def guardar_fichero(data, url):


    """
    Esta función guarda un DataFrame (data) en la dirección (url) especificada en formato .csv.

    Parameters:
    -----------------
    data (DataFrame): El DataFrame que queremos guardar.
    url (str): String con la dirección dónde queremos guardar el DataFrame.

    Returns:
    ------------
    n/a
    """

    
    data.to_csv(url, index=True)
    print(f'Archivo guardado en {url}')

In [96]:
url_clean_data='https://raw.githubusercontent.com/nuriancg/project-da-promo-H-module-3-team-1/master/HR_CLEAN_DATA.csv'
df = guardar_fichero(df,url_clean_data)

Archivo guardado en https://raw.githubusercontent.com/nuriancg/project-da-promo-H-module-3-team-1/master/HR_CLEAN_DATA.csv


## FUNCIONES

In [105]:
# Leer archivo
url_raw_data='https://raw.githubusercontent.com/nuriancg/project-da-promo-H-module-3-team-1/master/HR_RAW_DATA.csv'
df = leer_fichero(url_raw_data)

In [114]:
#Cambiar nombre columnnas
df = cambiar_nombre_col(df)
#df.columns = [normalizar_encabezados(columna) for columna in df.columns]

In [23]:
#Eliminar columnas
lista_col_eliminar=['Employee_Count','Monthly_Income','Over_18','Years_In_Current_Role','Same_As_Monthly_Income','Salary', 'Standard_Hours','Role_Departament', 'Number_Children']
df = eliminar_columnas(df,lista_col_eliminar)

In [24]:
#Crear columnas nuevas
lista_col_crear = ['Age_Group', 'Satisfaction_Group']
df = crear_columnas(df, lista_col_crear)

In [25]:
#Convertir a string
lista_col_to_str=['Daily_Rate','Department','Education_Field','Employee_Number','Distance_From_Home','Environment_Satisfaction','Gender','Performance_Rating','Remote_Work','Total_Working_Years','Work_Life_Balance' ]
df = to_string(df,lista_col_to_str)

In [26]:
#Cambio de ',0' por ''
lista_col_to_no_coma=['Daily_Rate','Employee_Number','Performance_Rating','Total_Working_Years','Work_Life_Balance']
df = to_no_coma(df,lista_col_to_no_coma)

In [27]:
#Cambio Booleanos
lista_col_to_booleanos = ['Attrition', 'Over_Time','Remote_Work']
df = to_bool(df,lista_col_to_booleanos)

In [28]:
#Otros cambios
lista_col_otros = ['Age','Business_Travel','Daily_Rate', 'Department','Distance_From_Home','Environment_Satisfaction','Gender','Marital_Status','Percent_Salary_Hike']
df = limp_otros(df,lista_col_otros)

/var/folders/g3/22l1jqy501nblpbhtydz9b0w0000gn/T/ipykernel_788/3791712289.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[col] = data[col].str.replace('$', '')


In [29]:
#Cambio a .title()
lista_col_to_title = ['Business_Travel', 'Education_Field','Job_Role','Marital_Status']
df = to_title(df,lista_col_to_title)


In [30]:
#Cambio nulos str a 99999
lista_col_int_nulos=['Daily_Rate','Employee_Number','Hourly_Rate','Performance_Rating','Total_Working_Years','Work_Life_Balance']
df = to_int_nulos(df,lista_col_int_nulos)

In [31]:
#CAmbio a int
lista_col_to_int=['Age','Daily_Rate','Distance_From_Home','Employee_Number','Environment_Satisfaction','Hourly_Rate','Job_Involvement','Monthly_Rate','Total_Working_Years','Performance_Rating','Work_Life_Balance']
df = to_int(df,lista_col_to_int)

In [32]:
#Cambio a bool
lista_col_to_bool=['Remote_Work']
df = to_bool(df,lista_col_to_bool)

In [33]:
# Int con .median()
lista_col_median = ['Performance_Rating','Work_Life_Balance']
df = to_median(df,lista_col_median)

In [34]:
# int con .mean()
lista_col_mean = ['Total_Working_Years']
df=to_mean(df,lista_col_mean)

In [35]:
# Categoricas to 'Unknown'
lista_col_unknown =['Business_Travel','Department','Education_Field','Marital_Status','Over_Time']
df = reemplazar_nulos_por_unknown(df, lista_col_unknown)

In [43]:
#Rellenar columnas
df = age_group(df,'Age_Group')
df = asignar_grupo(df, 'Job_Satisfaction')